In [2]:
import pandas as pd

dfData = pd.read_pickle('final_data.pkl')

In [3]:
dfData[84:].head()

,True,body_char_per_word,body_length,body_puncuation,body_sentiment_neu,body_sentiment_pos,body_statistic_ratio,body_vocab_rate,body_word_per_sent,resource,title_compounds,title_length,title_puncuation,title_sentiment_neu,title_sentiment_pos
84,0,6,2474,0.081649,0.816,0.109,0.004184,0.356790,93,21wire,"{'NNP': 0.6363636363636364, 'DT': 0.0909090909...",11,0.000000,1.000,0.0
85,0,5,287,0.101045,0.870,0.103,0.033233,0.622356,47,21wire,"{'NNP': 0.8, 'IN': 0.13333333333333333, 'NNS':...",15,0.066667,1.000,0.0
86,0,6,192,0.046875,0.911,0.029,0.069652,0.681592,40,Vanessa Beeley,"{'NNP': 0.6666666666666666, 'DT': 0.1333333333...",15,0.133333,0.859,0.0
87,0,5,306,0.120915,0.878,0.082,0.033803,0.605634,44,21wire,"{'NNP': 0.8666666666666667, 'IN': 0.0666666666...",15,0.133333,1.000,0.0
88,0,6,615,0.100813,0.855,0.092,0.013616,0.599092,66,21wire,"{'NNP': 0.6666666666666666, 'RB': 0.1666666666...",6,0.166667,0.595,0.0


In [4]:
import numpy as np

D = np.array(dfData[84:])
print(np.shape(D))
# Labels
Y = np.array([x[0] for x in D])
print(sum(Y))
X1 = D[:,3:9]
X2 = D[:,-4:]
# Features
X = np.concatenate((X1,X2),axis=1)
print(X)

(772, 15)
425
[[0.08164915117219078 0.816 0.109 ..., 0.0 1.0 0.0]
 [0.10104529616724739 0.87 0.103 ..., 0.06666666666666667 1.0 0.0]
 [0.046875 0.911 0.029 ..., 0.13333333333333333 0.859 0.0]
 ..., 
 [0.11834862385321102 0.796 0.156 ..., 0.3333333333333333 1.0 0.0]
 [0.1261335531739489 0.826 0.131 ..., 0.14285714285714285 1.0 0.0]
 [0.14732142857142858 0.83 0.125 ..., 0.2222222222222222 0.769 0.231]]


In [6]:
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

skf = StratifiedKFold(n_splits=10, random_state=None, shuffle=True)

C = np.zeros([2,2])

for train_index, test_index in skf.split(X, Y):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    # Train classifier
    clf = svm.SVC(probability=True)
    clf.fit(X_train, Y_train)  
    #SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    #decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
    #max_iter=-1, probability=False, random_state=None, shrinking=True,
    #tol=0.001, verbose=True)
    
    Y_pred = clf.predict(X_test)
    #print(clf.predict_proba(X_test))
    C += confusion_matrix(Y_test,Y_pred)
print(C)  

[[ 0.96291311  0.03708689]
 [ 0.96448815  0.03551185]
 [ 0.96286505  0.03713495]
 [ 0.92329282  0.07670718]
 [ 0.9556311   0.0443689 ]
 [ 0.96107626  0.03892374]
 [ 0.96291023  0.03708977]
 [ 0.96215081  0.03784919]
 [ 0.90365833  0.09634167]
 [ 0.96248547  0.03751453]
 [ 0.90311186  0.09688814]
 [ 0.96381618  0.03618382]
 [ 0.92679714  0.07320286]
 [ 0.96337511  0.03662489]
 [ 0.96302704  0.03697296]
 [ 0.9556311   0.0443689 ]
 [ 0.90350112  0.09649888]
 [ 0.96251683  0.03748317]
 [ 0.07087665  0.92912335]
 [ 0.96182272  0.03817728]
 [ 0.96505834  0.03494166]
 [ 0.9629746   0.0370254 ]
 [ 0.90301559  0.09698441]
 [ 0.96283167  0.03716833]
 [ 0.96287759  0.03712241]
 [ 0.96282504  0.03717496]
 [ 0.96287673  0.03712327]
 [ 0.95958047  0.04041953]
 [ 0.96013507  0.03986493]
 [ 0.96314346  0.03685654]
 [ 0.97039005  0.02960995]
 [ 0.09265414  0.90734586]
 [ 0.96357335  0.03642665]
 [ 0.1048876   0.8951124 ]
 [ 0.96398181  0.03601819]
 [ 0.08488307  0.91511693]
 [ 0.56277673  0.43722327]
 

In [68]:
sum(np.diag(C))/(sum(sum(C)))

0.92098445595854928

In [ ]:
def SVM_fit(self):
    import pandas as pd
    import numpy as np
    from sklearn import svm
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import confusion_matrix

    json_dat = [doc for doc in self._db.clean_data.find()]
    df_dat = pd.DataFrame(json_dat)
    df_dat_fit = df_dat.drop(['_id','body_length','resource','title_compounds','url'],axis=1)
    D = np.array(df_dat_fit)
    Y = D[:,0]
    X = D[:,1:]

    K = 10
    skf = StratifiedKFold(n_splits=K, random_state=None, shuffle=True)

    acc = []
    for train_index, test_index in skf.split(X, Y):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]

        # Train classifier
        #self.clf = svm.SVC(probability=True)
        self.clf.fit(X_train, Y_train)  

        Y_pred = clf.predict(X_test)

        Cfold = confusion_matrix(Y_test,Y_pred)
        self.classifier_performance['confusion'] += Cfold
        acc.append(sum(np.diag(Cfold))/sum(sum(Cfold)))
    
    acc = np.array(acc)
    self.classifier_performance['accuracy_mean'] = np.mean(acc)
    self.classifier_performance['accuracy_std'] = np.std(acc)

In [ ]:
def SVM_predict(self,url):
    
    _, clean_data, _ = self.data_getter(self,url,0)
    
    df_dat = pd.DataFrame(clean_data)
    
    Y = np.array(df_dat['True'])
    
    df_dat.drop(['_id','body_length','resource','title_compounds','url','True'], axis=1, inplace=True)
    
    X = np.array(df_dat)
    
    yProb = clf.predict_proba(X)
    return yProb